# Adding classification and tags

In this tutorial, we will:
- Create a Database Service, a Database, a Schema and one Table,
- Create a Classification,
- Create a Tag for the Classification and add it to the Table.

In [1]:
from metadata.ingestion.ometa.ometa_api import OpenMetadata
from metadata.generated.schema.entity.services.connections.metadata.openMetadataConnection import (OpenMetadataConnection, AuthProvider)
from metadata.generated.schema.security.client.openMetadataJWTClientConfig import OpenMetadataJWTClientConfig

## 1. Create a connexion to the Open Metadata server
 
In the server config clause, we must define the `server host and port`, the credential(in our case is a jwt token)

In [2]:
from creds import om_admin_token
server_config = OpenMetadataConnection(
    hostPort="http://datacatalog.casd.local/api",
    authProvider=AuthProvider.openmetadata,
    securityConfig=OpenMetadataJWTClientConfig(
        jwtToken=om_admin_token,
    ),
)
metadata = OpenMetadata(server_config)

In [3]:
# if it returns true, it means the connection is success 
metadata.health_check()

True

## 2. Create the required entities

In [4]:
from metadata.generated.schema.api.services.createDatabaseService import CreateDatabaseServiceRequest
from metadata.generated.schema.entity.services.connections.database.common.basicAuth import BasicAuth
from metadata.generated.schema.entity.services.connections.database.mysqlConnection import MysqlConnection
from metadata.generated.schema.entity.services.databaseService import (DatabaseConnection, DatabaseService, DatabaseServiceType,)
from metadata.generated.schema.api.data.createDatabase import CreateDatabaseRequest
from metadata.generated.schema.api.data.createDatabaseSchema import CreateDatabaseSchemaRequest
from metadata.generated.schema.api.data.createTable import CreateTableRequest
from metadata.generated.schema.entity.data.table import Column, DataType

# create db service
db_service = CreateDatabaseServiceRequest(
    name="test-db-service",
    serviceType=DatabaseServiceType.Mysql,
    connection=DatabaseConnection(
        config=MysqlConnection(
            username="db_login",
            authType=BasicAuth(password="db_name"),
            hostPort="http://db_url:1234",
        )
    ),
)

# when we create an entity by using function `create_or_update`, it returns the created instance of the query
db_service_entity = metadata.create_or_update(data=db_service)

# create db
db_entity_req = CreateDatabaseRequest(
    name="test-db",
    service=db_service_entity.fullyQualifiedName,
)

db_entity = metadata.create_or_update(data=db_entity_req)

# create schema
create_schema_req = CreateDatabaseSchemaRequest(
    name="test-schema", database=db_entity.fullyQualifiedName
)

# the create request will return the fqn(fully qualified name) of the created schema
schema_entity = metadata.create_or_update(data=create_schema_req)

# create table

table_a = CreateTableRequest(
    name="test_user",
    databaseSchema=schema_entity.fullyQualifiedName,
    columns=[Column(name="id", dataType=DataType.BIGINT,description="id of the user"),
             Column(name="age", dataType=DataType.INT,description="age of the user")],
)

table_a_entity = metadata.create_or_update(data=table_a)

## 3.create a classification entity

In [5]:
from metadata.generated.schema.api.classification.createClassification import CreateClassificationRequest
from metadata.generated.schema.api.classification.createTag import CreateTagRequest

classification_request=CreateClassificationRequest(
    name="TestClass1",
    description="Sample classification for test.",
)

metadata.create_or_update(classification_request)

Classification(id=Uuid(__root__=UUID('f001bd5f-02b2-4ce0-b4f4-0ad7fbc24873')), name=EntityName(__root__='TestClass1'), fullyQualifiedName=FullyQualifiedEntityName(__root__='TestClass1'), displayName=None, description=Markdown(__root__='Sample classification for test.'), version=EntityVersion(__root__=0.1), termCount=None, updatedAt=Timestamp(__root__=1719323707432), updatedBy='ingestion-bot', href=Href(__root__=AnyUrl('http://datacatalog.casd.local/api/v1/classifications/f001bd5f-02b2-4ce0-b4f4-0ad7fbc24873', scheme='http', host='datacatalog.casd.local', tld='local', host_type='domain', path='/api/v1/classifications/f001bd5f-02b2-4ce0-b4f4-0ad7fbc24873')), usageCount=None, changeDescription=None, deleted=False, provider=<ProviderType.user: 'user'>, disabled=None, mutuallyExclusive=False)